# Estudio sobre la tienda on-line de:

## [AMANTIS](https://www.amantis.net/)

## ¿Por qué este estudio?

Primero, este es un trabajo de análisis de datos y de predicción sobre una tienda on-line. Creo que las personas que veamos esta documentación somos adultas y lo suficientemente maduras para asumir que el sexo es parte de nuestra vida.

Por este motivo, el hacer un estudio sobre una tienda on-line erótica es lo mismo que realizarlo sobre una tienda de ropa, muebles, un supermercado,...

Segundo, yo soy usuario registrado de esta tienda on-line. No solo realizo compras para mí o para mi pareja sino también para otras personas. 
Es por este motivo el que recibo periódicamente correos de ofertas de esta tienda on-line y es aquí en donde entra mi motivación personal.

Los correos que envían a sus usuarios están basados en ofertas que lanzan (estas ofertas están basadas en rebajas) y son muy arbitrarias. Suelo recibir correos de ofertas de determinados productos de los que NO he realizado una compra del mismo o similares. 

Por este motivo, considero interesante poder realizar algún tipo de estudio sobre los productos que tengan en la tienda y poder dar una mejor experiencia al comprador.

## Consideraciones para el proyecto.

Este proyecto está estructurado en 4 partes:

- Obtención y tratamiento de datos de la página web, a partir de *Webscrapping* y su manipulación para poder tratarlos a través de las librerías de *pandas* y *numpy* y su almacenamiento en una *Base de datos Relacional*, dada la posibilidad de limitar los campos a realizar y la atomización de los datos.
   - A su vez se realizará algún tipo de visionado de los datos obtenidos para ver diversos aspectos a tener en cuenta.
- Generación de nuevas variables, *featuring engineering*, para poder dar una mejor visión de los productos que ofrecen en la tienda.
    - A través de la generación de estas nuevas variables, pretendemos dar un visionario más completo sobre la relación de las diversas variables entre sí y los usuarios.
    - Estas variables pueden ser obtenidas a través de *pandas*, *RegEx* y *NLP*.
- Análisis de la información obtenida y generada para ver la relación existente entre usuarios, productos, fechas...
   - Esta información será aplicada a partir de diversas queries de la base de datos a utilizar y con las librerías de *pandas*, *matplotlib* y *seaborn*.
- Generación de modelados de los datos obtenidos para establecer gustos de los usuarios y dar sugerencia de otros productos que tengan etiquetas similares.

    


## 1.- Obtención y tratamiento de los datos.

Esta parte del proyecto ya está presentada como un proyecto individual, por lo cual para seguir su desarrollo les remito al susodicho proyecto ([repositorio de Webscrapping](https://github.com/75Engel/Webscrapping_tienda_online)).

Aqui lo que vamos a presentar como detalle es el codigo final que se ha utilizado para la obtención de los datos.

A través de la librería para Webscrapping *Beautiful Soap* procederemos a extraer esta información y en determinados casos a tratarla para poder trabajar con ella.

Para ello cargaremos las librerías necesarias.

#### Estas serían las librerias importadas:

~~~~
from bs4 import BeautifulSoup as bs

import requests

import pandas as pd

import numpy as np

from datetime import datetime
~~~~

#### Estas serían las variables necesarias:

~~~
url_principal="https://www.amantis.net/"                        
url_secundaria=url_principal+'productos-amantis/'

date=str(datetime.today().strftime('%y%m%d'))
folder_ingest=r'.\Data'

ext=r'.csv'
scrape='_scrape'
file_product=r'\productos'
file_user=r'\usuarios'
file_comment=r'\comentarios'
file_price=r'\precios'
file_tag=r'\tags'
file_ingest_product=file_product+scrape
file_ingest_comment=file_comment+scrape

product_ingest=folder_ingest+file_ingest_product+'_'+date+ext
comment_ingest=folder_ingest+file_ingest_comment+'_'+date+ext
product_engineer=folder_ingest+file_product+'_'+date+ext
tag_engineer=folder_ingest+file_tag+'_'+date+ext
price_engineer=folder_ingest+file_price+'_'+date+ext
comment_engineer=folder_ingest+file_comment+'_'+date+ext
user_engineer=folder_ingest+file_user+'_'+date+ext

pages= np.arange(1,25)
~~~

#### Este sería el código para la obtención de la información:

~~~~
'''Listas a generar con la información de los productos'''

lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
id=[]
comentarios=[]

'''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}
diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}

'''Listas para generar la información de los comentarios'''
id_comment=[]
comments=[]
date=[]
ratio=[]
users=[]
comment=[]

print("Empezando a recoger datos de las paginas")
for page in pages:
    
    if page == 1:
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
print("Terminando de recoger los datos de los links de las paginas\nEmpezando a generar las listas de los productos")

for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))

    
    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)

    datos = list(zip(date_comments_product,rating, user_comments_product,comments_product ))
    comentarios.append(datos)

for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]

print("Realizando la ingenieria de los datos\nEliminando duplicados de productos")
productos=pd.DataFrame(diccionario_datos_productos)
noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']

print("Tratando los comentarios")

comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)
comentarios=pd.DataFrame()
diccionario={"id":id_comment,"comments":comments}

for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
    for i in n_comments:
        id_comment.append(id_product)
        comments.append(i)


for j in range(len(diccionario['comments'])):
    date.append(diccionario['comments'][j][0])
    ratio.append(diccionario['comments'][j][1])
    users.append(diccionario['comments'][j][2])
    comment.append(diccionario['comments'][j][3])


comentarios['ID']=pd.Series(id_comment)
comentarios['DATE']=pd.Series(date)
comentarios['RATIO']=pd.Series(ratio)
comentarios['USERS']=pd.Series(users)
comentarios['COMMENT']=pd.Series(comment)

noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]

h=input("Quieres salvar los datos?").upper()
if h=="SI":
    noduplicated_product.to_csv(product_ingest,header=True,index=False)           
    noduplicated_comments.to_csv(comment_ingest,header=True,index=True)           
~~~~

## 2.- Ingeniería de datos.

Quiero hacer notar que parte de este punto se mostro en el anterior proyecto, pero en este repositorio aparece todo el proceso de generación de datos que hemos realizado para obtener nuevas variables que son importantes para analizar las características de los productos, las tendencias de los usuarios y de este modo poder realizar un análisis en profundidad de la información obtenida y el modelado que queremos plantear.

Los dataframes que se obtienen son los siguientes:

*Dataframe productos:*

![Dataframe productos](./Resources/image/dataframe_productos.JPG)


*Dataframe comentarios:*

![Dataframe comentarios](./Resources/image/dataframe_comentarios.JPG)

Es por ello que en el **dataframe de productos** vamos a generar diversos dataframes con la información más sintetizada posible separando los precios del resto.

Mientras que el nombre, la descripción y las características son valores que no variarán en el tiempo y, por lo tanto, pueden permanecer en un **dataframe de productos**, no pasa lo mismo con los precios pueden fluctuar cada vez que se realiza una extracción y por lo tanto hay que dejarlos aparte con una fecha de referencia de cuando se han tomado, creando un **dataframe de precios**.

Además crearemos un dataframe completamente nuevo a partir de la columna descripción usando *keywords* existentes en esta columna para asignar *tags* a cada producto. Este dataframe lo llamaremos **dataframe de tags**.

Realmente no es necesario crear este dataframe aparte del dataframe de productos, pero lo hemos realizado así para que sea más visible la información resultante del mismo.

En el **dataframe de comentarios**, vamos a realizar una atomización de los valores de los usuarios para que el tratamiento de la información sea más manejable.

Es por ello que crearemos un **dataframe de usuarios**, más o menos arbitrario dada la imposibilidad de asignar correctamente los usuarios, y un **dataframe de comentarios**, donde haremos una pequeña manipulación del campo *date* para poder realizar posteriormente los análisis precisos.

### 2.1- Creando dataframe de usuarios.

Vamos a discriminar en función de si un nombre de usuario a escrito un comentario sobre un producto, si vuelve a aparecer este nombre se genere un nombre nuevo, que será el mismo con un dígito para distinguirlos.

Es cierto que esta forma de presentar los datos a través de la página web aumenta el anonimato de las personas que lo compran, ya que lo habitual es que se repitan nombres sin coincidir con un mismo usuario (¿cuántos José hay en España?¿y en el mundo?)

No es posible hacer un seguimiento de qué productos compran las personas de este modo, pero vamos a realizar un "simulacro" de cómo se debería de realizar.

El modo correcto sería usando id de usuarios, lo que solo es posible por parte de la empresa o si se utilizarán nicks para que aparecieran los usuarios que realizan los comentarios por su nick.

El código a continuación muestra como se realiza la sustitución "in situ" en el dataframe.
~~~
nombre_count = {}
count = {}

for i, row in df_comments.iterrows():
    id = row['ID']
    nombre = row['USERS']
    
    if id not in count:
        count[id] = {}
        
    if nombre in count[id]:
        count[id][nombre] += 1
        nuevo_nombre = f"{nombre}_{count[id][nombre]}"
        df_comments.loc[i, 'USERS'] = nuevo_nombre
    else:
        count[id][nombre] = 1
~~~

Posteriormente creamos una diccionario a partir de una lista creada con los valores únicos del campo *USERS* y con los índices de los usuario de esta lista.

Con este diccionario agregamos el campo *ID_USERS* con un mapeo en el *dataframe de comentarios* y, posteriormente creamos el *dataframe de usuarios*.

~~~
'''mapeo de usuarios'''
lista_users=df_comments['USERS'].unique()                  
mivalor = [ x for x in range(len(lista_users))]             
lista_users=list(lista_users)                                
lista_users_code = {k: v for k, v in zip(lista_users, mivalor)}   
print(lista_users_code)
df_comments['ID_USERS']= df_comments['USERS'].map(lista_users_code)

'''creación del dataframe de usuarios'''
USERS=pd.DataFrame()
USERS['ID_USERS']=df_comments['ID_USERS']
USERS['USERS']=df_comments['USERS']
USERS.drop_duplicates(subset='ID_USERS', keep='first',inplace=True)

'''reemplazamos el campo USERS por ID_USERS''' 
col = df_comments.pop('ID_USERS')
df_comments.drop(columns=['USERS'],inplace=True)
df_comments.insert(loc= 4 , column= 'ID_USERS', value= col)
~~~

### 2.2- Creando dataframe de comentarios.

Ya, una vez hemos reemplazado el campo USERS por ID_USERS, nos queda modificar la información que existe en el campo *DATE* para adaptarla a un formato de fecha que podamos trabajar.

Recordemos que la información que aparece en este campo tiene esta formulación: miércoles 25 enero, 2023.

Y queremos que tenga esta formulación: 2023-01-25.

Para ello usaremos el método string para obtener las posiciones 1, 2 y 3 y reemplazar la posición 2 con un mapeado por el número de mes correspondiente.

~~~
'''Este es el diccionario con el cual realizaremos el mapeado''' 
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

'''Con esta porción de código realizaremos la extracción y el mapeado correspondiente'''
df_comments['DAY']=df_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
df_comments['MONTH']=df_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
df_comments['YEAR']=df_comments['DATE'].str.split(' ').str.get(-1).astype('Int64')
df_comments['MONTH']=df_comments['MONTH'].map(dm_mapping)

'''Aqui regeneramos el campo DATE con la información obtenida'''
df_comments['DATE'] = pd.to_datetime(df_comments.iloc[:,-3:])
df_comments=df_comments.iloc[:,:-3]
~~~


Una vez que ya hemos generados los dataframes relacionados con los comentarios, vamos a proceder a mostrar como generamos los dataframes relacionados con los productos.

### 2.3- Creando dataframe de precios.

Para ello generamos un nuevo campo, que denominados *FECHA* que hace referencia a la fecha en la cual se hace este proceso de ingeniería, con un formato de yymmdd y generamos un nuevo dataframe con los campos *ID_PRODUCT*, *REGULAR_PRICE* y *DISCOUNT_PRICE*. 

En este proceso lo primero que hacemos es reestructurar el dataframe, ya que ID está separado del resto de los campos que necesitamos y luego recortamos el dataframe.

~~~
'''Generamos el valor de DATE que vamos a incorporar en el dataframe, así como creamos las variables que hacen referencia a los campos que vamos a dejar en el dataframe''' 
date=str(datetime.datetime.today().strftime('%y%m%d'))
col_1 = dataframe.pop('REGULAR_PRICE')
col_2=dataframe.pop('DISCOUNT_PRICE')
col_3=dataframe['ID']

'''Colocamos nuevamente las variables en las posiciones que queremos dejar'''
dataframe.insert(loc= 1 , column= 'ID_PRODUCT', value= col_3)
dataframe.insert(loc= 2 , column= 'REGULAR_PRICE', value= col_1)
dataframe.insert(loc= 3 , column= 'DISCOUNT_PRICE', value= col_2)
df_prices=dataframe.iloc[:,:4]

'''Añadimos el campo FECHA'''
df_prices['FECHA']=date
~~~

En los 2 siguientes puntos vamos a centrarnos en los campos *NAME* e *INFO* que tienen gran información de texto referente al producto (principalmente el segundo campo).

Del campo *NAME* nos interesa reducir el volumen de información que existe, por lo cual lo separaremos en 2 campos.

El campo *INFO* también lo vamos a separar en 2 campos, del que usaremos principalmente el primer campo que crearemos, aunque el segundo campo sí tiene información que podría ser evaluable. 

### 2.4- Creando dataframe de productos.

Aquí mostramos como creamos los campos *PRODUCT* y *SLOGAN* a partir del campo *NAME*, para crear los campos *CHARACTERISTICS* y *DESCRIPTION* haremos un procedimiento similar.

El campo *NAME* tiene diversas peculiaridades que hemos intentado subsanar con la primera línea de código, ya que *PRODUCT* y *SLOGAN* se encuentran separados por un guión, pero este guión también aparece dentro del texto que queremos dejar en estos campos por lo cual nos vemos obligados a realizar una pequeña sustitución antes de realizar el split.

~~~
dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)
dataframe['PRODUCT'] = dataframe['PRODUCT'].str.strip()
dataframe['SLOGAN'] = dataframe['SLOGAN'].str.strip()
~~~

En el campo *INFO* también existe algún caracter que hay que sustituir, y lo trabajamos de un modo similar. En este caso, es la existencia de salto de línea (\r).

Una vez realizada estas transformadas, recolocamos los campos que vamos a dejar en el dataframe de productos y eliminamos el resto.

~~~
'''Definimos las variables con los nombres de los campos'''
col_1 = dataframe.pop('PRODUCT')
col_2=dataframe.pop('SLOGAN')
col_3=dataframe.pop('DESCRIPTION')
col_4=dataframe.pop('CHARACTERISTICS')

'''Eliminamos los campos primigenios con los que hemos creado los nuevos campos'''
dataframe.drop(columns=['NAME'],inplace=True)
dataframe.drop(columns=['INFO'],inplace=True)

'''Posicionamos los nuevos campos y reducimos el dataframe'''
dataframe.insert(loc= 1 , column= 'PRODUCT', value= col_1)
dataframe.insert(loc= 2 , column= 'SLOGAN', value= col_2)
dataframe.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
dataframe.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
df_product=dataframe.iloc[:,:6]
~~~
 

### 2.5- Creando dataframe de tags.

Esta es la parte más importante del proceso de ingeniería de datos que tiene el proyecto ya que vamos a generar un dataframe nuevo a partir del campo *DESCRIPTION*, dejando también los campos *ID* y *PRODUCT* para poder relacionarlo con el dataframe de producto.

Básicamente vamos a crear un *diccionario*, cuya clave será el nombre del nuevo campo creado y cuyo valor será una lista con keywords que evaluaremos con **SpaCy** para asignar un True/False en el caso que aparezca alguna de las keywords.

Este es un punto crítico y de continua evaluación, ya que hay que confirmar periódicamente que los productos se etiquetan correctamente.

El diccionario que vamos a utilizar es:

{**'amenities'**: ['lubricante', 'limpiador', 'preservativo', 'condon'],

 **'anal'**: ['anal'],

 **'BDSM'**: ['latex', 'bdsm', 'arnes', 'strap', 'cera', 'ligadura', 'cuerda', 'cuero', 'sumision', 'dominacion', 'latigo', 'watenberg', 'posicionador', 'mordaza'],

 **'femenino'**: ['mujer', 'femenino', 'vaginal', 'clitoris', 'dildo'],

 **'masculino'**: ['hombre', 'masculino'],

 **'juguetes'**: ['dildo', 'plug', 'vibrador', 'masturbador', 'cabezal', 'estimulador', 'plugs', 'bolas chinas', 'funda extensora', 'bomba de vacio'],

 **'lenceria'**: ['babydoll', 'body', 'camiseta', 'corse', 'corset', 'diadema', 'encaje', 'falda', 'joyeria', 'lenceria', 'lencero', 'malla', 'media', 'panties', 'pantis', 'ropa', 'sujetador', 'vestido'],
 
 **'muebles'**: ['chaise', 'columpio', 'moqueta', 'silla', 'sillon', 'sillones']}

Este diccionario está creado a partir de ficheros .pickle donde están guardadas cada lista y cuyo nombre es el valor de la clave. Se crea a partir de la función que existe en la carpeta [src\Utils](Utils/functions.py).

A partir de las funciones: **aplicar_funcion_a_columna** y **extraer_lemas** se generan las nuevas variables.

~~~
def extraer_lemas(text):
    '''
    Función que recibe un texto como entrada y devuelve una lista de los lemas de las palabras que se encuentran en el texto. 
    
    La lista de lemas resultante solo contiene las palabras que son alfabéticas, es decir, que no contienen caracteres numéricos o especiales.

    - Inputs:
        - text (str):           Texto del cual se quieren extraer los lemas.

    - Outputs:
        - lemas (list):         Lista de los lemas de las palabras alfabéticas que se encuentran en el texto.
    '''
    doc = nlp(text)
    lemas = [token.lemma_ for token in doc if token.is_alpha]
    return lemas
~~~
Con esta función se lemmatiza cada texto existente en la columna *DESCRIPTION*.

~~~
def aplicar_funcion_a_columna(df, Dict_pickle, nombre_listas, columna="DESCRIPTION"):
    '''
    Función que genera columnas en un dataframe en función de la presencia de determinados lemas en una columna específica de un Dataframe.

    Por defecto, la columna es DESCRIPTION.

    - Inputs: 
        - df (Dataframe):       Dataframe a tratar.
        - Dict_pickle (dict):   Diccionario con listas
        - nombre_listas (list): Lista proveniente del Diccionario Dict.
        - columna (str):        Columna del dataframe a tratar.
    
    - Outputs: 
        - df (Dataframe):       Dataframe tratado.
    '''

    # Carga la lista desde el diccionario
    lista = Dict_pickle[nombre_listas]

    # Aplica la función a la columna del DataFrame
    df[nombre_listas] = df[columna].apply(lambda x: any(lematizado in lista for lematizado in extraer_lemas(x)))

    return df
~~~
Con esta función se buscan los diferentes valores de cada par clave-valor del diccionario en el texto lemmatizado de *DESCRIPTION* y crea una nueva columna cuyo nombre es el valor de la clave y cuyo valor en cada registro es True/False en el caso que se encuentre alguno o ninguno de los posibles valores que están incluidos en la lista.

## 3. Generación de base de datos.

Con la información que hemos obtenido y creado en los puntos anteriores vamos a proceder a guardarla en una base de datos para, posteriormente, poder aplicarla en otros procesos.

Me he decidido en utilizar una base de datos utilizando *SQLite*, tras consultar con expertos en Full-Stack y orientarme sobre la operatibilidad de la misma al tener guardarla en un repositorio de GitHub.

La estructura de la base de datos es:

![Estructura_tablas](./Resources/image/Relacion_tablas.jpg)

A partir de aquí generaremos las diferentes *tablas* y *views* que consideremos oportunas para el manejo de datos.

Para los que no conozcan la importancia del uso de *views* en bases de datos, les recomiendo que miren en este [enlace](https://www.linkedin.com/feed/update/urn:li:activity:7058051069079093248?utm_source=share&utm_medium=member_desktop) donde hablo sobre el uso de views.  

#### Estas serán las librerias necesarias para esta parte del proyecto:

~~~~
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3
from Utils import functions as f
~~~~

También utilizaremos una función que tenemos guardada en Utils\functions.py, que utilizamos para generar Pandas.Dataframe a partir de SQL.

~~~
def sql_query(query,cursor):
    '''
    Función que genera un dataframe a partir de una orden a la base de datos.

    - Inputs:
        - query (str):          Query realizada a la base de datos.
        - cursor (cursor):      Cursor de conexión a la base de datos.

    - Outputs:
        - Dataframe (Dataframe): Dataframe de la query requerida.
    '''

    # Ejecuta la query
    cursor.execute(query)

    # Almacena los datos de la query 
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)
~~~

### 3.1 Generación de BBDD.

Crearemos la Base de Datos en la carpeta de Resources desde la que podremos acceder en cualquier momento.

Directamente  podemos crearla generando la conexión a la misma, el nombre de la misma será **online_shop.db**
~~~
conn = sqlite3.connect("Resources/online_shop.db")
cursor = conn.cursor()
~~~

#### Creamos las tablas a continuación:

~~~
query_product = '''
CREATE TABLE PRODUCT  (
    ID INT (3),
    PRODUCT VARCHAR (100),
    SLOGAN VARCHAR (100),
    DESCRIPTION VARCHAR (250),
    CHARACTERISTICS VARCHAR (250),
    URL VARCHAR (50),
    PRIMARY KEY (ID)
)
'''

query_prices = '''
CREATE TABLE PRICES  (
    ID INT (3),
    ID_PRODUCT INT (3),
    REGULAR_PRICE INT (3),
    DISCOUNT_PRICE INT (3),
    DATE_DOWNLOAD VARCHAR (100),
    PRIMARY KEY (ID),
    FOREIGN KEY (ID_PRODUCT) REFERENCES PRODUCT (ID)
)
'''

query_comment = '''
CREATE TABLE COMMENT  (
    ID_COMMENT INT (5),
    ID_PRODUCT INT (3),
    DATE VARCHAR (100),
    RATIO INT (2),
    ID_USERS INT (3),
    COMMENT VARCHAR (250), 
    PRIMARY KEY (ID_COMMENT),
    FOREIGN KEY (ID_PRODUCT) REFERENCES PRODUCT (ID),
    FOREIGN KEY (ID_USERS) REFERENCES USERS (ID)
)
'''

query_tags = '''
CREATE TABLE TAGS  (
    ID INT (3),
    JUGUETES INT(1),
    MUEBLES INT(1),
    LENCERIA INT(1),
    MASCULINO INT(1),
    FEMININO INT(1),
    ANAL INT(1),
    BDSM INT(1),
    AMENITIES INT(1),
    PRIMARY KEY (ID),
    FOREIGN KEY (ID) REFERENCES PRODUCT (ID)
)
'''

query_users = '''
CREATE TABLE USERS  (
    ID INT (5),
    USERS VARCHAR (50),
    PRIMARY KEY (ID)
)
'''

'''Ejecutamos las queries para generar las tablas'''

cursor.execute(query_product)
cursor.execute(query_prices)
cursor.execute(query_comment)
cursor.execute(query_tags)
cursor.execute(query_users)
~~~

#### Creamos las vistas:

~~~
view_1='''
CREATE VIEW COMMENTS_PER_PRODUCT
SELECT PRODUCT.PRODUCT,COMMENT.DATE,COMMENT.RATIO ,COMMENT.COMMENT 
FROM COMMENT 
JOIN PRODUCT ON PRODUCT.ID=COMMENT.ID_PRODUCT
'''

view_2='''
CREATE VIEW NUMBER_TAGS_PER_PRODUCT 
SELECT PRODUCT.PRODUCT, PRODUCT.ID,
SUM(CASE WHEN TAGS.JUGUETES = TRUE THEN 1 ELSE 0 END),
COMMENT.RATIO ,COMMENT.COMMENT 
FROM COMMENT 
JOIN PRODUCT ON PRODUCT.ID=COMMENT.ID_PRODUCT
'''

'''Ejecutamos las queries para generar las vistas'''

cursor.execute(view_1)
cursor.execute(view_2)
~~~

### 3.2 Carga de datos en la Base de datos,

A partir de los 5 dataframes generados, cargaremos directamente los datos de modo masivo en cada tabla.

El proceso es general para cada tabla, modificando el número de campos de la misma.

Pondremos como ejemplo el caso de la tabla *PRODUCT*:

~~~
lista_products= df_products.values.tolist()                                                         #   Convertimos todo el dataframe en una lista

cursor.executemany("INSERT INTO PRODUCT VALUES (?,?,?,?,?,?)", lista_products)                      #   Salvamos directamente todos los datos en la tabla PRODUCTS

                                                                                                    #   Habrá tantos interrogantes como campos tenga la tabla donde se guarda.
~~~

### 3.3 Cerramos la Base de Datos.

Dejamos la base de datos cerrada a la espera de tener que acceder a ella, bien para realizar actualizaciones en la misma con nuevos ficheros, bien para realizar llamamientos para visualización o carga de Dashboards.

~~~
conn.commit()
cursor.close()
conn.close()
~~~

## 4. Visualización de datos.


A partir de los datos guardados en la base de datos, vamos a realizar un visionado de los diferentes puntos que podemos evaluar.

Veremos que información disponemos en relación con los productos y los usuarios.

Todo el proceso de tratamiento de la información que disponemos la realizaremos a partir de comandos SQL que pasaremos a un dataframe para realizar el visionado.

#### Estas serán las librerias necesarias para esta parte del proyecto:

~~~~
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
~~~~

### 4.1 PRICE.

En relación con este punto, vamos a realizar diversos estudios relacionados con los precios que tienen los productos y los descuentos que se aplican.

Sobre esta tabla realizaremos visionados referentes a los precios más altos, con y sin descuentos realizados.

#### 4.1.1 APPLIED DISCOUNT.

**¿Qué porcentaje de productos tienen un descuento aplicado?**

Creo que esta pregunta es muy relevante dado que al visitar la página web te encuentras con muchos productos con descuento.

Es por ello que vamos a ver qué cantidad de productos tienen y qué cantidad de productos no tienen descuento.

Para ello vamos a generar un nuevo parámetro que será *PERCENTUAL DISCOUNT* cuyo valor será: ([DISCOUNT_PRICE]-[REGULAR_PRICE])/[REGULAR_PRICE] y que compararemos con el *PRICE* del producto.

Al tener solo 2 posibles valores vamos a visualizar esto con un *Donut chart*.

~~~
plt.figure(figsize=(10,5))
plt.title("PERCENTAGE OF OFFERS", fontsize=16)

label=['WITH_DISCOUNT','WITHOUT_DISCOUNT']                                  

'''Definimos un pieplot en blanco para dejar solo una circunferencia de la estrechez indicada'''
e=0.7
my_circle=plt.Circle( (0,0), e, color='white')            

query='''
SELECT PRODUCT.PRODUCT AS NAME, CAST(DATE_DOWNLOAD as INT) as DATE, REGULAR_PRICE,
[DISCOUNT_PRICE]-[REGULAR_PRICE] AS DISCOUNT,
ABS([DISCOUNT_PRICE]-[REGULAR_PRICE])/[REGULAR_PRICE]*100 as PERCENTUAL_DISCOUNT,           
CASE WHEN [DISCOUNT_PRICE] = [REGULAR_PRICE] THEN 0 ELSE 1 END AS WITH_DISCOUNT
FROM PRICES  
JOIN PRODUCT ON PRODUCT.ID=PRICES.ID_PRODUCT
ORDER BY DISCOUNT DESC
'''                                                        
discount_df=f.sql_query(query,cursor)

data = discount_df.groupby(['DATE'])["WITH_DISCOUNT"].value_counts()

plt.pie(data.values,
        labels=label,
        autopct='%1.2f%%',
        textprops={'fontsize': 8},
        startangle=90, counterclock=True)

plt.ylabel(data.index[0][0],fontsize=12)
plt.yticks()
plt.xticks(rotation=90)
p=plt.gcf()
p.gca().add_artist(my_circle)
~~~

![Relacion_descuento](./Resources/Graphics/Percentage_offers.png)

Como vemos en el grafico solo un 0,5% de los productos no tienen descuentos, lo que quiere decir que todo el inventario que se vende tiene un descuento aplicado.

Si está situación es habitual cabría la posibilidad de plantearse que sigue una política continua de *descuento permanente* en el producto y que el "precio sin descuento" no se aplica nunca.

**¿Qué porcentaje de descuento tienen los productos?**

Hemos visto que prácticamente todos los productos tienen descuentos, pero ¿cómo está distribuido el porcentaje aplicado en los productos?

Es lo que vamos a ver a continuación.

Para ello vamos a realizar un *boxplot* para ver las distancias intercuantílicas, por un lado, y un *swarmplot* para ver cuantitativamente la distribución de los precios.

~~~
plt.figure(figsize=(10,5))
plt.title("DISCOUNT DISTRIBUTION")
plt.ylabel("DISCOUNT %")
plt.xlabel("")

query='''
SELECT PRODUCT.PRODUCT AS NAME, CAST(DATE_DOWNLOAD as INT) as DATE, REGULAR_PRICE, DISCOUNT_PRICE,
[DISCOUNT_PRICE]-[REGULAR_PRICE] AS DISCOUNT,
CAST(ABS([DISCOUNT_PRICE] - [REGULAR_PRICE]) / [REGULAR_PRICE] * 100 AS INT) AS PERCENTUAL_DISCOUNT
FROM PRICES 
JOIN PRODUCT ON PRODUCT.ID=PRICES.ID_PRODUCT
ORDER BY PERCENTUAL_DISCOUNT DESC
'''                                           

sns.boxplot(data=f.sql_query(query,cursor)['PERCENTUAL_DISCOUNT'],color="pink")
sns.swarmplot(f.sql_query(query,cursor)['PERCENTUAL_DISCOUNT'],color='black')

'''Definiendo el outlier inferior'''
outlier_1 = np.percentile(f.sql_query(query,cursor)['PERCENTUAL_DISCOUNT'], 1.1)        
plt.axhline(y=outlier_1, xmin=0, xmax=1,color='r', linewidth= 1.5,linestyle="-.")
~~~

![Relacion_descuento](./Resources/Graphics/discount_distribution.png)

El umbral inferior de los valores por debajo de 1,1% sean outliers.

De hecho es algo que visualmente se ve, en general los productos están por encima de 20%. Muy distribuidos entre el 20% y el 80% de descuento.

Se puede ver claramente que el número de productos que serían outliers serían 7, siendo algunos de estos productos los que no tienen descuento.

En general y por termino medio los productos tienen aproximadamente un descuento que ronda la mitad de su precio. Es por ello la siguiente cuestión.

#### 4.1.2 PRICE DISTRIBUTION.

**¿Cómo se distribuyen los precios de los productos?**

Vamos a realizar un estudio tanto del precio rebajado como el precio sin rebajar, queremos ver como se comportan los precios en sí mismos para sacar conclusiones.

Para ello vamos a realizar dos subplots con un *histograma* en cada uno.

Como bien podemos imaginar hay una gran variedad de precios, por ello lo primero que realizaremos es una modificación en los precios para poder agrupar mejor los valores.

Para ello vamos a generar un nuevo parámetro que será *ABSOLUTE_PRICE* cuyo valor será: CAST(DISCOUNT_PRICE/10 AS INT)*10.

Dejaremos los valores en decenas de unidad para aguparlos.

~~~
fig, axes = plt.subplots(1, 2, figsize=(15,5), sharey=True)
axes[0].set_position([0,1.5,0.5,0.300])
axes[1].set_position([0.5,1.5,0.2,0.300])

label=["Discount Price Distribution","Regular Price Distribution"]

'''PLOT 0'''

query_1='''
SELECT PRODUCT.PRODUCT AS NAME,CAST(DISCOUNT_PRICE/10 AS INT)*10 as ABSOLUTE_PRICE
FROM PRICES 
JOIN PRODUCT ON PRODUCT.ID=PRICES.ID_PRODUCT
'''                                
abs_disc_price_df=f.sql_query(query_1,cursor)
sns.distplot(abs_disc_price_df['ABSOLUTE_PRICE'],bins=50,color="teal", ax=axes[0])


'''PLOT 1'''

query_2='''
SELECT PRODUCT.PRODUCT AS NAME, CAST(REGULAR_PRICE/10 AS INT)*10 as ABSOLUTE_PRICE
FROM PRICES 
JOIN PRODUCT ON PRODUCT.ID=PRICES.ID_PRODUCT
'''                                
abs_reg_price_df=f.sql_query(query_2,cursor)
sns.distplot(abs_reg_price_df['ABSOLUTE_PRICE'],bins=50,color="olive", ax=axes[1])

for i in range (0,2):
    axes[i].set_ylabel('Density')
    axes[i].set_xlabel('PRICE')
    axes[i].set_title(label[i])
    axes[i].set_xticklabels(axes[i].get_xticklabels(), fontsize=10)

fig.suptitle("Prices_Distribution_study",fontsize=15)
fig.tight_layout() 
~~~

![Histograma_precios](./Resources/Graphics/Prices_Distribution_study.png)

En relación con el **primer subplot**, que hace referencia al precio rebajado, hay una gran variedad de densidad de productos que oscilan entre los precios de 0-10 € hasta los productos que llevan a los 100 €, aunque principalmente los precios se agrupan más en torno entre los 0 € y los 30 €.

Además los precios rebajados no superan los 350 € como vemos al final de su histograma.

En relación con el **segundo subplot**, que hace referencia al precio sin rebajar, el rango de precios es mayor, llegando hasta 170 € y con una densidad más repartida.

Además los precios pueden llegar hasta los 1000 € como vemos al final de su histograma.

### 4.2 COMMENTS.

Los datos que tenemos de comentarios cubre un amplio abanico en el **tiempo**, es por ello que podemos hacer un estudio sobre la evolución en el tiempo de este campo y ver cuando se realizan más comentarios y cuando menos.

Además los comentarios van acompañados de una **evaluación** de los productos comprados por lo que podemos hacer un estudio de este aspecto.

Finalmente también podemos hacer uso de procesos de NLP para hacer una evaluación del **sentimiento** de los usuarios sobre los productos comentados y ver el grado de satisfacción con los mismos.

Procedemos pues:

#### 4.2.1 TIME.

Dado que tenemos un campo de fechas en la realización de los comentarios, vamos a proceder a ver cuando se realizan los usuarios comentarios sobre los productos comprados.

Disponemos de datos sobre los años, los meses, los días del mes, los días de las semana en que se realizan estos comentarios, por lo que podemos ver pautas de comportamiento.

En este punto, tenemos que realizar la suposición que el comentario se realiza en el momento de la compra del producto y que todos los productos que se compran son comentados automáticamente por los usuarios, lo cual no deja de no ser cierto.

Sería interesante en este sentido disponer de la información real de compras de estos productos para conocer por ejemplo qué ratio de conversión de compra a comentario existe y ver si la página web tiene una comunidad viva que aporte.

También hay que tener en cuenta que existen una serie de productos que tienen compras repetitivas en el tiempo, pero que por este motivo es de pensar que solo tendrán un único comentario por usuario que realiza la compra, serían productos que hemos asignado como *AMENITIES*.

El análisis que vamos a realizar solo contemplará la información de tiempo de *AÑO*, *MES* y *DIA DE LA SEMANA*.

Como el proceso tratamiento del campo *DATE* es el mismo y solo se realiza una agrupación de la fecha, vamos a mostrar la información en 3 subplots de barras.

~~~
fig, axes = plt.subplots(1, 3, figsize=(15,5), sharey=True)
axes[0].set_position([0,1.5,0.5,0.300])
axes[1].set_position([0.5,1.5,0.2,0.300])
axes[2].set_position([1.5,1.5,0.2,0.300])

label=["YEAR","MONTH","DAY OF THE WEEK"]

'''PLOT 0'''

query='''
select count (COMMENT) AS COMENTARIOS, strftime ('%Y',DATE) as YEAR 
from COMMENT 
GROUP BY YEAR
'''
total=f.sql_query(query,cursor)
total.loc[len(total)] = [0,2006]
total.iloc[:,1]=total.iloc[:,1].astype(int)
sns.barplot(x=total['YEAR'],y=total['COMENTARIOS'],color="skyblue", ax=axes[0])

'''PLOT 1'''

query='''
select count (COMMENT) AS COMENTARIOS, strftime ('%m',DATE) as MONTH 
from COMMENT 
GROUP BY MONTH
'''   
total=f.sql_query(query,cursor)
total.iloc[:,1]=total.iloc[:,1].astype(int)
sns.barplot(x=total['MONTH'],y=total['COMENTARIOS'],color="steelblue", ax=axes[1])

'''PLOT 2'''

query='''select count (COMMENT) AS COMENTARIOS, strftime ('%w',DATE) as DAY_WEEK 
from COMMENT 
GROUP BY DAY_WEEK
'''   
total=f.sql_query(query,cursor)
total.iloc[:,1]=total.iloc[:,1].astype(int)
total['DAY_WEEK']=total['DAY_WEEK'].map(dicc_dia)
sns.barplot(x=total['DAY_WEEK'],y=total['COMENTARIOS'],color="teal", ax=axes[2])

for i in range (0,3):
    axes[i].set_ylabel('')
    axes[i].set_xlabel('')
    axes[i].set_title(label[i])
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45, fontsize=7)

fig.suptitle("COMMENTS BY DATE OF STUDY",fontsize=15)
fig.tight_layout() 
~~~
![Histograma_comentarios](./Resources/Graphics/Comments_per_date.png)

En este punto podemos realizar varias **conclusiones**:

1) En relación con el **año**:

    Hay un crecimiento de comentarios hasta aproximademente el 2015, para después estancarse. También se observa un nuevo pico en el 2020, puede ser que sea por el COVID-19, pero el resto de los años hay una constante de comentarios en torno a los 900 comentarios.

    Los datos que se muestran de 2023 son más bajos, dado que la extracción de la información no es completa de ese año.

2) En relación con los **meses**:

    Se observa mucha estabilidad en los datos, habiendo picos de comentarios en enero (coincidiendo con Reyes) y en Septiembre (coincidiendo con la vuelta en la mayoría de los casos de las vacaciones).

    El tema de las vacaciones puede ser algo interesante de evaluar en el sentido de si los usuarios tienen pareja con niños o si realizan sus compras durante los meses estivales anteriores y comentan a la vuelta y desvirtua en parte este análisis.

3) En relación con el **día de la semana**:

    Se observa que los días de la semana con menores comentarios sean los días del fin de semana y en general se agrupan en los días entre semana.

    Esto puede ser porque al comercializar un producto de ocio, esté se compre y se comente fuera del fin de semana, que es cuando más podría ser utilizado al no ser productos de consumo inmediato.

#### 4.2.2 RATINGS.

Hemos visto como se distribuye la presencia de comentarios en el tiempo, pero **¿cómo son vistos los productos por los usuarios?**

Esta es la pregunta que nos hacemos ahora.

Tenemos una fuente directa de esta informacion en la página web, las estrellas que les dan los usuarios que van desde 1 a 5.

También podríamos usar una fuente indirecta, la perdida de usuarios por la página web, pero esto es algo que no vamos a entrar en este proyecto.

Vamos a realizar un *histograma* de valoraciones para ver el promedio de valoración de cada producto.
~~~
plt.figure(figsize=(10,5))
query='''
SELECT PRODUCT,avg(RATIO) as PROM 
FROM COMMENTS_PER_PRODUCT 
group BY PRODUCT 
ORDER BY avg(RATIO) DESC
'''
total=f.sql_query(query,cursor)
sns.histplot(data=total,bins=100)
plt.xticks()
plt.ylabel("Nº PRODUCTS")
plt.xlabel("RATING")
plt.title("DISTRIBUTION OF PRODUCT RATIOS")
file=str(r'\Resources/Graphics/ratios_per_product')
plt.legend(labels=[])
~~~
![Distribucion_comentarios](./Resources/Graphics/ratios_per_product.png)

Podemos ver que prácticamente todos los productos tienen una valoración superior a 4, con una alta cantidad de productos con una valoración de 5 estrellas.

¿Cabe la posibilidad que entonces los productos inferiores a 4 estrellas sean outliers?¿y los de 5 estrellas?

Esto es lo que vamos a ver a continuación.

Para ello vamos a realizar un *boxplot* para evaluar los cuartiles existentes.

~~~
plt.figure(figsize=(10,5))
query='''
SELECT PRODUCT,avg(RATIO) as PROM 
FROM COMMENTS_PER_PRODUCT 
group BY PRODUCT
ORDER BY avg(RATIO) DESC
'''
sns.boxplot(data=f.sql_query(query,cursor)['PROM'])
outlier = np.percentile(f.sql_query(query,cursor)['PROM'], 5.5)
plt.axhline(y=outlier, xmin=0, xmax=1,color='r', linewidth= 1,linestyle="-.")
plt.ylabel("RATING")
plt.title("DISTRIBUTION OF PRODUCT RATIOS")
~~~
![Distribucion_comentarios](./Resources/Graphics/Outliers_per_product.png)

Podemos ver claramente que los productos con una valoración inferior a 3.75 estrellas aproximadamente se podrían considerar outliers.

También que los productos con una media de valoración de 5 estrellas no son outliers.

Los datos obtenidos en estudios de estos datos indican que por debajo de 3,75 estrellas son 23 productos y mientras que las que tienen 5 estrellas son 129, lo que representan un 5.15 % y 28.86 % de los productos del inventario de la página web.

Que más de un 25 % de los productos tengan la valoración máxima implica que, por lo general, todos los usuarios que hacen comentarios están contentos con los productos que compran.

#### 4.2.3 SENTIMENT.

Este punto es un aspecto que dada la información que hemos obtenido en el apartado anterior, no vamos a valorar en este proyecto.

Como hemos visto la mediana se encuentra en 4.75 puntos y hay mucha acumulación de comentarios con la máxima puntuación por lo que el sentimiento de los comentarios va a ser bueno.

Dejamos para un posible futuro la posibilidad de realizar este estudio pero en los comentarios de menor valoración.

### 4.3 Users.

#### 4.3.1 NUMBER OF USERS IN THE COMMUNITY.

Este punto puede ser crítico.

**¿Cuántos usuarios han realizado un comentario al menos en el intervalo de tiempo que lleva existiendo la página web?**

Lo podemos ver directamente con esta consulta:

~~~
query='''
SELECT COUNT(COMMENT)AS nº_COMMENTS, USERS.USERS AS USERS
FROM COMMENT 
JOIN USERS
ON COMMENT.ID_USERS=USERS.ID
GROUP BY USERS
ORDER BY nº_COMMENTS DESC
'''
users=f.sql_query(query,cursor)
print("El número de usuarios que han comentado en algún momento un producto es", len(users))
~~~

La respuesta a esta pregunta son 2021 usuarios. 2021 usuarios que han realizado un número indefinido de comentarios.

Pero, **¿cuántos de estos usuarios han realizado únicamente 1 comentario?**

~~~
user_comment_once=users[users['nº_COMMENTS']==1]
print("El número de usuarios que han comentado unicamente una vez es",len(user_comment_once))
~~~

La respuesta a esta pregunta es 827 usuarios. 

Esto representa casi la mitad de los usuarios que han realizado un comentario, por lo que se puede decir que o bien los usuarios solo realizan un único comentario o deja de comprar en esta página web.

#### 4.3.2 DISTRIBUTION OF USERS BY YEAR.

**¿Cómo se comportan los usuarios en el tiempo?**

Para proseguir con este estudio vamos a incorporar a este análisis univariante una segunda variable, el año de realización del comentario.

Vamos a ver la evolución de los usuarios en el tiempo y ver si los usuarios se mantienen en el tiempo.

~~~
query='''
SELECT COUNT(COMMENT)AS nº_COMMENTS, strftime ('%Y',DATE) as YEAR, USERS.USERS AS USERS
FROM COMMENT 
JOIN USERS
ON COMMENT.ID_USERS=USERS.ID
GROUP BY USERS,YEAR
ORDER BY YEAR asc
'''

comments_users_per_year=f.sql_query(query,cursor)
comments_users_per_year = comments_users_per_year.pivot(index='USERS', columns='YEAR', values='nº_COMMENTS')
comments_users_per_year = comments_users_per_year.fillna(0)
comments_users_per_year['TOTAL_COMMENTS'] = comments_users_per_year.sum(axis=1)                                             
comments_users_per_year = comments_users_per_year.sort_values(by='TOTAL_COMMENTS', ascending=False)                         
comments_users_per_year
~~~

y vamos a considerar el TOP 50 de usuarios para poder realizar un estudio *sólo* sobre aquellos usuarios que son activos en la comunidad y ver la información en un *heatmap*.

![heatmap_usuarios](./Resources/Graphics/N_TOP_Commented_Users_by_YEAR.png)

Aqui vemos 2 situaciones diferentes:
1) Hay casos puntuales donde un usuario aparece fugazmente, realiza una gran cantidad de comentarios y luego desaparece.
2) Hay casos que realizan unos pocos comentarios cada año, pero son fieles a la comunidad y aportan a ella.

En cualquier caso, no hay que olvidar que el mayor problema para este análisis es que hemos generado la variable *USERS* a partir de los nombres de los comentarios y, por lo tanto, si dos usuarios distintos tienen el mismo nombre y comentan dos productos distintos, este proceso los consideran como si fueran el mismo usuario.


### 4.4 TAGS.

Las etiquetas o TAGS de los productos son algo inherente a cada producto, es un modo de poder asignar características comunes a los mismos y poder clasificar así los mismos.

Lo bueno que tiene esto es que podemos clasificar a los usuarios en relación a su comportamiento de compra de los mismos.

Aquí vamos a considerar 2 preguntas:

#### 4.4.1 TAGS BY PRODUCT.

**¿Qué tipo de productos tenemos en nuestro inventario?**

Es una pregunta fácil, pero con esto podemos saber qué tipo de productos son más demandados y cuales no.

Para evaluar esta pregunta vamos a realizar un *lollipop* con cada uno de los posibles TAGS que existen.

Como cada TAG es una columna independiente, crearemos una variable que tenga en cuenta el valor total de los TAGS. 

~~~
plt.figure(figsize=(10,5))
query = '''
SELECT 'JUGUETES' AS TAGS, SUM(JUGUETES) AS valor FROM TAGS
    UNION
    SELECT 'MUEBLES' AS TAGS, SUM(MUEBLES) AS valor FROM TAGS
    UNION
    SELECT 'LENCERIA' AS TAGS, SUM(LENCERIA) AS valor FROM TAGS
    UNION
    SELECT 'MASCULINO' AS TAGS, SUM(MASCULINO) AS valor FROM TAGS
    UNION
    SELECT 'FEMININO' AS TAGS, SUM(FEMININO) AS valor FROM TAGS
    UNION
    SELECT 'ANAL' AS TAGS, SUM(ANAL) AS valor FROM TAGS
    UNION
    SELECT 'BDSM' AS TAGS, SUM(BDSM) AS valor FROM TAGS
    UNION
    SELECT 'AMENITIES' AS TAGS, SUM(AMENITIES) AS valor FROM TAGS
ORDER  BY valor DESC;
'''

total=f.sql_query(query,cursor)
total.iloc[:,1]=total.iloc[:,1].astype(int)
plt.hlines(total['TAGS'],xmin=0,
           xmax=total['valor'],)
plt.plot(total['valor'], total['TAGS'], "o")
plt.yticks(total['TAGS'])
plt.xlabel("Nº PRODUCT")
plt.ylabel("TAGS")
plt.title("Nº PRODUCT PER TAG")
~~~
![tags_product](./Resources/Graphics/N_Tags_product.png)

#### 4.4.2 TAGS BY USERS.

**¿Qué tipo de productos buscan los usuarios?**

Pretendemos tener una visión sobre los tipos de productos que compran y valoran los usuarios.

Del mismo modo que en el punto anterior, procederemos a relacionar los *TAGS* con los *USERS* en la consulta.

![tags_product](./Resources/Graphics/N_Tags_users.png)


No sabemos si tiene relación, pero si comparamos las dos graficas frente a frente, podemos observar que el comportamiento de las TAGS son similares.

![tags_product](./Resources/Graphics/TAGS_study.png)

Lo que podemos decir en relación con estos datos que vemos es que no hay muchos usuarios que comenten productos dentro de *AMENITIES*, bien porque son pocos productos los que llevan esta etiqueta, bien porque son productos que como solo se comentan una vez por usuario (no es que no lo permita la página web, pero nadie comenta varias veces un mismo producto).

También vemos que hay muchos productos y muchos usuarios que suelen comentar productos con la etiqueta *ANAL*, teniendo además en cuenta que al generar este tag, se usa una lista de las más pequeñas.

### 4.5 CONCLUSIONES.

Tras esta evaluación de los datos recabados creo que podemos tener claro varios puntos:

1) Por norma general los productos tienen descuento sobre el precio oficial, además esta rebaja oscila en torno al 50 %, es decir cuesta la mitad de lo que dicen que cuesta.

    Los precios de esta manera suelen oscilar entre 0 € y 100 €.

2) Los periodos en los cuales los productos son comentados (comprados) son los meses de enero y septiembre y preferiblemente a mediados de semana.

    No hay ninguna modificación en cuanto al volumen a lo largo de los años, los comentarios se han quedado estancados en torno a los 900 comentarios/año.

3) Los productos tienen una muy buena valoración por los usuarios que realizan los comentarios.

    Prácticamente los productos tienen un 4 o un 5 de valoración y solo un 5% de los productos tienen baja valoración

4) A pesar de contabilizar más de 2000 usuarios que han realizado al menos un comentario, más del 40% solo han realizado un único comentario.

    Esto implica una baja actividad en la comunidad por parte de los usuarios, pero no sabemos si la baja actividad lleva consigo que hayan abandonado la comunidad o no.
    
    También podemos ver entre los 50 principales usuarios, que tenemos registrados en la base de datos, que se tratan de usuarios con actividad en la comunidad, ya que lo habitual es que comenten varios productos a lo largo de los años.

5) En relación con las TAGS podemos ver la poca presencia de comentarios en *AMENITIES* lo cual es implicito al hecho que son productos de compra repetitiva, pero que no se comentan varias veces por el mismo usuario.

    Hay una relación directa entre los productos comentados y los usuarios, una TAG con muchos productos lleva implicito muchos usuarios que lo comentan y viceversa.

    